# Jupyter Setup

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2

# Setup

In [3]:
# Import modules
import json
import math
from simply_tiles.tms import TileMatrixSet

# Die 2D Tile Matrix

*Die folgenden Ausführungen basieren auf der [TileMatrixSet 2.0](https://docs.opengeospatial.org/DRAFTS/17-083r3.html) Spezifikation der OGC (derzeit im Draft)*

Eine einzelne 2D Tile Matrix (im folgenden auch zu TM verkürzt) ist ein Gitter über eine gegebene Bounding Box. Um sie begrifflich von der Bounding Box der Geometrien abzugrenzen, die zu Vector Tiles verarbeitet werden sollen, wird sie fortan als „Extent“ bezeichnet. Laut TMS 2.0 wird das Gitter durch folgende Komponenten vollständig definiert:

* `extent` - *bounding box die in ein Kachelgitter aufgeteilt werden soll*
* `matrixWidht` und `matrixHeight` - *Zahl der Kacheln entlang X und Y, die den Extent vollständig bedecken sollen.*
* `tileSize` - *für X und Y in CRS Einheiten. Die Kachelgrößen folgen aus der zuvor gewählten Kachelzahl*
* `origin` - *Obere linke Ecke des Extents. Von hier aus werden die Kacheln entlang x (links nach rechts) und y (oben nach unten) gezählt als 0 basierter Index gezählt*

Weiterhin gibt es auch **tile matrix limits**, mit denen angegeben wird, in welchem Bereich der Tile Indizes Geometrien vorhanden sind. Die sind sowohl für Clients als auch Server nützlich. So muss der verwendete Kachelgenerator nicht alle theoretisch möglichen Kacheln erzeugen sondern nur solche, in denen Geometrien zu sehen sind.

# Einzelne Kacheln der 2D Tile Matrix als Rasterkacheln rendern

Um ein Raster-Cahce zu erzeugen, muss jede Kacheln einzeln gerendert werden. Dafür wird sie in Pixel unterteilt. 
So ergibt sich ein weiteres „Gitter“ über jeder einzelnen Kachel, das als **extrapolated device grid** bezeichnet wird. Mit "device" ist ein „visualization device“ zum Rendern gemeint. Die Gitterräume werden als **grid cells** bezeichnet (fortan als Zellen übersetzt). Höhe und Breite einer (Pixel-)Zelle ergeben sich aus folgenden Vorgaben:

`tileWidth` und `tileHeight` - *Zahl der Pixel entlang X und Y. Meist identisch entlang X und Y, was die TMS Spec aber nicht vorschreibt*
`PixelScaleDenominator` - *Verhältnis der Pixelgröße zur realen Größe in Metern*

Die tatsächliche Pixelgröße eines Rendering Devices ist im Vorfeld nicht bekannt. Als Referenz nutzt die OGC daher einen quadratischen „Standardpixel“ von **0.28mm * 0.28mm** (wie schon in den WMS, SE und WMTS Spezifikationen). 

Was in der TMS Spec nicht direkt ersichtlich wird (zumindest für den Laien): Wozu braucht es den Standardpixel überhaupt? Würde das Rendering Device nicht ohnehin mit der tatsächlichen Pixelgröße rechnen? 

Des Rätsels Lösung: Der Standardpixel wird im Grunde als Näherungswert eingesetzt, um die (von Device zu Device variierende) Zellengröße einer gegebenen Tile Matrix in einen räumlichen Maßstab umzurechnen (und umgekehrt).

# Tile Matrix Sets und ihre Maßstäbe

Um ein Raster-Cache zu erzeugen, muss zunächst bestimmt werden, welche Maßstäbe für die gegebene Anwendung überhaupt benötigt werden. 
Pro Maßstab wird dann eine passende Tile Matrix gebildet und entsprechende Kacheldateien erzeugt. Es braucht also ein „Set“ an Kachelmatrizen: das **Tile Matrix Set**.

* Jede Tile Matrix eines Sets bekommt laut TMS Spec einen „alphanumerischen Identifier“
* Das ist im Grunde äquivalent zum Konzept „Zoomlevel“, nur das die OGC das allgemeiner fassen will. Da in beiden Fällen Integer IDs vergeben werden, ist diese Differenzierung eher nebensächlich.

Übrigens: Innerhalb eines TMS kann die `tileWidth` entlang der Y-Achse variiert werden. Gängig ist z.B. der Ansatz, zu den Polen hin breitere Kacheln zu verwenden. Dafür wird das Tile Matrix Set um einen graduellen Parameter erweitert. Soweit ich die Spec richtig deute, wird das nur für Projektionen mit extremerer Verzerrung sowie in 3D Anwendungen genutzt. Für 2D Tiles in gängigen Projektionen sollte das nicht weiter relevant sein. In [T-Rex](https://t-rex.tileserver.ch/) wurde der Parameter bisher nicht implementiert. Da das Projekt als maßgebliche Referenz für diese Python-Implementierung dient, wird der Parameter auch hier nicht weiter betrachtet.

## Das TMS WebMercatorQuad und seine "krummen" Maßstäbe

Wenn man sich nun das gängige TMS [WebMercatorQuad](https://docs.opengeospatial.org/DRAFTS/17-083r3.html#web-mercator-quad-tilematrixset-definition-httpwww.opengis.netdeftilematrixsetogc1.0webmercatorquad) anschaut, stellt sich die Frage: Warum die krummen Maßstäbe? 

In [4]:
# Zur Illustration die Maßstäbe für Zoomstufe 0, 1 und 2: 
scale_denominator_level0 = 559082264.0287178
scale_denominator_level1 = 279541132.0143589
scale_denominator_level2 = 139770566.0071794

Warum sollte ein Anwender für die erste Zoomstufe 0 einen unintuitiven Wert wie *1 : 559082264.0287178* wählen? 

Der Grund hierfür ist simpel: Im Falle von WebMercatorQuad folgen die Maßstäbe aus der Kachellung, nicht umgekehrt! Die Zahlen sind schlichtweg eine logische Folge aus der Entscheidung, den gesamten Extent der Projektion EPSG:3857 in eine **Bildpyramide** aufzuteilen: [Illustration](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fcdn.geotribu.fr%2Fimg%2Farticles-blog-rdp%2Fdivers%2FTilePyramid.jpg&f=1&nofb=1)

Im TMS 2.0 Spec ist das Konzept näher beschrieben. Im Grunde ist das die einfachste, intuitivste Weise, ein TMS zu definieren:
* Nimm das gesamte CRS (in diesem Fall quadratisch) als Extent für alle Tile Matritzen im TMS
* Die erste Kachel umfasst den gesamten Extent und bekommt den Zoomlevel 0.
* Für jedes weitere Zoomlevel, teile Kachellänge und Kachelbreite durch 2 (= Verfierfachung der Kachelanzahl)
* Benutze zum Rendern 256x256 Pixel (`tileWidth` = `tileHeight`) pro Kachel 

So kommt es auch, dass sich die Maßstäbe mit jeder weiteren Zoomstufe halbieren:

In [5]:
(scale_denominator_level0 / 2) == scale_denominator_level1

True

## Zellgröße berechnen

Ausgehend von den geschilderten Regeln lässt sich der Maßstab für Zoomstufe 0 wie folgt bestimmen.
Zunächst brauchen wir die korrespondierende Zellengröße:

In [6]:
# Extent des EPSG 3857 (in Metern)
extent = {
    "xmin" : -20037508.3427892,
    "ymin" : -20037508.3427892,
    "xmax" : 20037508.3427892,
    "ymax" : 20037508.3427892
}

# Länge bzw. Breite der quadratischen Kachel in CRS Einheiten bei Zoomstufe 0 (entspricht in diesem Fall dem Erdumfang)
tileSpan = extent["xmax"] - extent["xmin"] 

# Anzahl Pixel entlang X und Y, in die die Kachel unterteilt wird
tileHeight, tileWidth = 256, 256 

# Berechnung der Zellengröße
cellSize = tileSpan / 256 
cellSize 

156543.03392804062

Eine Pixellänge / Pixelbreite misst also ca. `156543` Meter bei Zoomstufe 0!

Die Zellengröße wird auch als „Resolution Level“ bezeichnet. Präziser bleibt jedoch der Begriff „CellSize“ aus der TMS Spec. Schließlich ist hier nicht die Auflösung im Sinne einer Bildschirm Pixelzahl gemeint. Eher eine räumliche Auflösung, wie in diesen [Esri Blogartikel](https://desktop.arcgis.com/de/arcmap/10.3/manage-data/raster-and-images/cell-size-of-raster-data.htm) beschrieben.

## Maßstab berechnen

Die Zellengröße ist im Grunde schon ein Maßstab: Meter pro Pixel!
Um den Maßstab auf eine beliebige Einheit (Meter, Fuß, Grad) zu generalisieren, muss die Referenzgröße eliminiert werden. Dafür brauchen wir die „physikalische“ Größe des Pixels im Bildschirm. Hier kommt der zuvor erwähnte "Standardpixel" ins Spiel:

In [7]:
standardPixelSize = 0.00028 # in Metern

cellSize / standardPixelSize # Einheitsloser Maßstab

559082264.0287166

Wenn das zu Grunde liegende CRS in Grad oder Fuß bemessen ist, so hätte eine cellSize die Einheit Grad/Pixel bzw. Fuß/Pixel. 
Hier muss die cellSize zunächst in Meter umgerechnet werden, bevor das Teilen durch die "Standardpixelgröße" die Einheit gänzlich eliminiert.

## Unstimmigkeiten bei der Rundung

Am Rande: Die korrespondierende Angabe zur Zellengröße aus der WebMercatorQuad Tabelle im TMS 2.0 Draf weicht minimal vom obigen Beispiel ab.
Teilt man jenen Wert durch die Standard Pixelgröße entsteht wiederum ein minimal abweichender Wert vom angegebenen Maßstab.
Hier herrschen noch Unstimmigkeiten beim Handling von Rundungsfehlern. Auch bei T-Rex finden sich [rundungsbasierte Abweichungen](https://github.com/t-rex-tileserver/t-rex/blob/master/tile-grid/src/grid.rs).

In [8]:
cellSize_from_spec = 156543.0339280410
cellSize_from_spec / standardPixelSize 

559082264.0287179

## Zellgrößen und Maßstäbe abseits des Äquators

Abgesehen von der "wahren" Pixelgröße gibt es noch einen weiteren Aspekt, der die Herleitung von „wahren“ Maßstäben verkompliziert: Die projektionsspezifische, geometrische Verzerrung. Bei der Mercator Projektion beispielsweise wird die Verzerrung zu den Polen größer. Folglich gibt es zur Bestimmung des Maßstabs eine [Korrektur](https://docs.microsoft.com/en-us/bingmaps/articles/understanding-scale-and-resolution), bei der der Breitengrad einbezogen wird:

`cellSize * cos(latitude)`

Die Zellengrößen und die daraus abgeleiteten Maßstäbe aus der WebMercatorQuad Definition gelten nur für den Äquator bzw. den Breitengrad 0:

In [9]:
cellSize * math.cos(0)

156543.03392804062

In der Tabelle zum TMS „WorldCRS84Quad“ macht die TMS 2.0 Spec sogar explizit, dass die angegebenen Zellengrößen nur am Äquator gültig sind. Prinzipiell müsste sich dieser Hinweis in jeder TMS Tabelle wiederfinden. Zudem braucht jede Projektion vermutlich eine eigene Korrekturrechnung. Für das Erzeugen von Kacheln aus Vektorgeometrien ist das jedoch nebensächlich.

## Zusammenfassung

Startet man wie bei WebMercatorQuad mit einem "Kachel-Algorithmus", entsteht für jede Zoomstufe zunächst eine definierte  Kachelgröße bzw. `tileSpan`. Zusammen mit der Pixelzahl ergibt sich dann die Zellengröße und schließlich der Maßsstab:

**Kachelgröße + Pixelzahl entlang X und Y --> Zellengröße <--> Maßstab**

Die formalisierte TMS Definition dreht diese Logik im Grunde nur um:

**Maßstab <--> Zellengröße --> Pixelzahl entlang X und Y + Kachelgröße**

Da sich Zellengröße und Maßstab jeweils voneinander ableiten lassen, genügt im Grunde nur einer von beiden Parametern.
Bei T-Rex wird ein Custom TMS beispielsweise nur mittels Zellgrößen (aka. „Resolutions“) definiert.
Bei GeoServer kann man zwischen beiden Größen wählen.

Weil die wahre Pixelgröße unbekannt ist, ist der Maßstab im TMS eine Näherung an den „wahren“ Maßstab, der sich aus dem Rendering Device ergibt. Der Rückgriff auf einen Standardpixel dient vermutlich dazu, eine Vergleichbarkeit zwischen unterschiedlichen TMS Definitionen herzustellen. Allerdings bedeutet es auch, dass Anwender mit Hilfe eines TMS keine exakten Maßstäbe vorgeben können!

# Kachelgitter aus TMS Parametern ableiten

Sollen Geometrien auf Kacheln verteilt werden, stellt sich zuerst die Frage: Auf welchen Kacheln wären überhaupt Geometrien zu sehen? Ausgehend von der Bounding Box muss das für jede Zoomstufe getrennt ermittelt werden. Das TMS Spec schlägt dafür Pseudocode vor (siehe Annex I). Hier eine simple Python Implementierung:

```
# Konstante um rundungsbasierte Abweichungen bei Dezimalzahlen
EPSILON = 0.0000001

# Auszug aus der Funktion tms.tile_limits():
limits = {
    "tileMinCol": math.floor((bbox_xmin - tile_matrix_xmin) / tilespan_x + EPSILON),
    "tileMaxCol": math.floor((bbox_xmax - tile_matrix_xmin) / tilespan_x - EPSILON),
    "tileMinRow": math.floor((tile_matrix_ymax - bbox_ymax) / tilespan_y + EPSILON),
    "tileMaxRow": math.floor((tile_matrix_ymax - bbox_ymin) / tilespan_y - EPSILON)
}
```

## Vollständige Weltkarte in WebMercatorQuad

Als erstes laden wir alle relevanten TMS Parameter und instantiieren damit ein Objekt der Klasse `TileMatrixSet`. Diese ist mit allen wichtigen Formeln ausgestattet. Analog zu T-Rex werden die Zellengrößen (resolution level) zur TMS-Definition herangezogen. Die Herleitung von Maßsstäben (am Äquator) aus den Zellengrößen ist ebenfalls implementiert (lediglich zur Illustration)

In [10]:
# TMS Definition aus einer json Datei lesen
with open('data/WebMercatorQuad.json', mode="r") as json_data_file:
    tms_definition = json.load(json_data_file)

# TileMatrixSet Objekt basierend auf WebMercatorQuad instantiieren
tms = TileMatrixSet(**tms_definition)

# Die ersten 3 Zellengrößen und die daraus abgeleiteten Maßstäbe:
for z in [0,1,2]:
    cell_size_z = tms.cell_sizes[z]
    scale_z = tms.scale_denominator(z)
    print(cell_size_z, scale_z)

156543.033928041 559082264.0287178
78271.5169640205 279541132.0143589
39135.75848201025 139770566.00717944


Zur Abbildung der gesamten Welt in WebMercatorQuad braucht es den gesamten EPSG:3857 `extent` als Bounding Box:

In [11]:
bbox = tms.extent # als bbox wird der gesamte extent verwendet
bbox

{'xmin': -20037508.342789248,
 'ymin': -20037508.342789248,
 'xmax': 20037508.342789248,
 'ymax': 20037508.342789248}

Sodann werden die Werte `tile_matrix_xmin` und `tile_matrix_ymax` benötigt. Hierbei handelt es sich um den `origin` der Kachel-Koordinaten. Üblicher Weise wird die obere linke Ecke des CRS verwendet. Welche Ecke als Referenz gilt, ist prinzipiell beliebig. Bei T-Rex lässt sich z.B. auch "bottom left" in einer Custom TMS auswählen. Der Einfachheit halber wird in dieser Implementierung nur die obere linke Ecke angeboten. Wichtig ist nur, dass origin und die "Referenzecke" vom extent übereinstimmen. Aus diesem Grund muss in dieser Implementierung kein separates Koordinatenpaar angegeben werden (analog zu T-Rex). Das TMS Objekt entnimmt die Werte direkt aus dem Attribut `extent`. Trotz Redundanz müssen im XML und JSON Encoding der TMS Spec die origin Koordinaten explizit angegeben werden. 

In [12]:
tms.origin # statt expliziter Koordinaten wird in dieser Implementierung nur der String "top_left" zur Explizierung des origin genutzt

tile_matrix_xmin, tile_matrix_ymax = tms.extent["xmin"], tms.extent["ymax"]
tile_matrix_xmin, tile_matrix_ymax

'top_left'

(-20037508.342789248, 20037508.342789248)

Bleiben noch die Kacheldimensionen bei gegebener Zoomstufe: `tilespan_x` und `tilespan_y`. Wie bereits erläutert können diese aus der definierten Zellengröße und `tileWidth` bzw. `tileHeight` ermittelt werden:

In [19]:
z = 0 # Beispiel Zoomstufe zur Illustration

tilespan_x = tms.tile_width * tms.cell_sizes[z]
tilespan_y = tms.tile_height * tms.cell_sizes[z]
tilespan_x, tilespan_y

(40075016.685578495, 40075016.685578495)

Jetzt sind alle Zutaten vorhanden, um ausgehend von einer bounding box und einer Zoomstufe die Kachelindizes zu ermitteln, innerhalb derer Geometrien vorhanden sind:

In [14]:
# Kachelindex Limits für diverse Zoomstufen (Illustration)
for z in [0, 4, 10]:
    tms.tile_limits(bbox, z)

{'tileMinCol': 0,
 'tileMaxCol': 0,
 'tileMinRow': 0,
 'tileMaxRow': 0,
 'matrixWidth': 1,
 'matrixHeight': 1}

{'tileMinCol': 0,
 'tileMaxCol': 15,
 'tileMinRow': 0,
 'tileMaxRow': 15,
 'matrixWidth': 16,
 'matrixHeight': 16}

{'tileMinCol': 0,
 'tileMaxCol': 1023,
 'tileMinRow': 0,
 'tileMaxRow': 1023,
 'matrixWidth': 1024,
 'matrixHeight': 1024}

## Beliebige Bounding Box erzeugen

Meist werden Tilecaches nur für einen Teilbereich der gesamten Weltkarte erzeugt.
Solange die Caches ein und dasselbe TMS verwenden, lassen sie sich in GIS Clients besonders einfach übereinanderlegen:
Bestimmte Kacheln (z.B. die Kachel mit den Indizes bzw. "Kachekoordinaten" x=0, y=1, z=1) verweisen immer auf identische Koordinatenbereiche.

Würde ein GIS-Client eine gegebene Bounding Box in ausgewählten Zoomstufen darstellen, würde er die anzufragenden Kacheln nach gezeigtem Prinzip ermitteln. Zur Illustration:

In [15]:
bbox = {
    "xmin": 50000,
    "ymin": 50000,
    "xmax": 100000,
    "ymax": 100000,  
} # in EPSG 3857

for z in [0, 4, 10]:
    tms.tile_limits(bbox, z)

{'tileMinCol': 0,
 'tileMaxCol': 0,
 'tileMinRow': 0,
 'tileMaxRow': 0,
 'matrixWidth': 1,
 'matrixHeight': 1}

{'tileMinCol': 8,
 'tileMaxCol': 8,
 'tileMinRow': 7,
 'tileMaxRow': 7,
 'matrixWidth': 1,
 'matrixHeight': 1}

{'tileMinCol': 513,
 'tileMaxCol': 514,
 'tileMinRow': 509,
 'tileMaxRow': 510,
 'matrixWidth': 2,
 'matrixHeight': 2}

## "Kachelkoordinaten" wieder in CRS Koordinaten umrechnen

Wie bisher dargelegt, lassen sich mittels TMS und einer gegebenen Bounding Box alle Kachelindizes bestimmen, auf denen Geometrien vorhanden sind.
Doch um entsprechende Kacheln zu erzeugen, müssen die Indizes wieder in eine Bounding Box umgerechnet werden. Nur so lassen sich Kacheln und die zu "kachelnden" Zielgeometrien zusammenbringen. Wie in der Einleitung erwähnt wird die Bounding Box einer Kachel als `envelope` bezeichnet. Zur Ermittlung des envelope hat die TMS Spec ebenfalls Pseudocode parat. Anbei wieder die Beispielimplementierung in Python:

```
# Auszug aus TileMatrixSet.tile_envelope():

envelope = {
            "xmin": tile_col * tilespan_x + tile_matrix_minx,
            "ymin": tile_matrix_maxy - (tile_row + 1) * tilespan_y,
            "xmax": (tile_col + 1) * tilespan_x + tile_matrix_minx,
            "ymax": tile_matrix_maxy - tile_row * tilespan_y
        }
```

Zur Illustration hier der envolpe zu einer von 4 Kacheln, die sich aus der Bounding Box des vorausgehenden Beispiels bei Zoomstufe 10 ergeben:

In [17]:
z=10
x=513
y=509
tms.tile_envelope(x, y, z)

{'xmin': 39135.75848200917,
 'ymin': 78271.51696402207,
 'xmax': 78271.51696402207,
 'ymax': 117407.27544603124}

# Rechteckige Kacheln und Extents

Der TMS Pseudocode erlaubt zwar unterschiedliche tileWidth und tileHeight Parameter, allerdings gibt es nur EINE einheitliche Zellengröße für X und Y.
Hier nochmal die Formel zur Ableitung von Kachellänge (tile_span_y) und Kachelbreite (tile_span_x) in CRS Einheiten aus gegebenen TMS Parametern:

```
tileSpanX = tileWidth * cellSize
tileSpanY = tileHeight * cellSize
```

*Hinweis: Analog zum TMS Spec wird hier CamelCase benutzt, während die Python Implementierung auf snake_case zur Variablenbenennung setzt*

Bei einer fixen Pixelgröße von 0.28mm und einer fixen Zellengröße lassen sich also rechteckige Kacheln definieren, wenn `tileWidth` != `tileHeight` ausfällt.
In den TMS Beispielen der Spec sind tileWidth und tileHeight jedoch stets identisch (vgl. Abschnitt "Common TileMatrixSet definitions").
Aus der Spec geht nicht hervor, wann und warum beide Parameter voneinander abweichen sollten. Das erschließt sich wohl erst, wenn man tiefer in das Thema Rasterdaten einsteigt.

Übrigens spricht nichts dagegen, einen rechteckigen Extent in quadratische Kacheln zu unterteilen. Zu einer oder mehreren Seiten des Extents würden die Kacheln dann schlicht über den Extent hinausragen. Im T-Rex Beispiel für ein Custom TMS passiert genau das. Zur Illustration wurde das Beispiel auch in die vorliegende Python Implementierung aufgenommen:

In [27]:
with open('data/TrexCustomTMS.json', mode="r") as json_data_file:
    tms_definition = json.load(json_data_file)

custom_tms = TileMatrixSet(**tms_definition)

Zur Illustration hier die Weite und Länge des Extents (SRID=2056)

In [62]:
print("srid: ", custom_tms.srid)
print("units: ", custom_tms.units)

custom_extent = custom_tms.extent
extent_width = custom_extent["xmax"] - custom_extent["xmin"]
extent_height = custom_extent["ymax"] - custom_extent["ymin"]
extent_width, extent_height

srid:  2056
units:  meters


(480000.0, 320000.0)

Wenn wir den gesamten Extent "kacheln" wollen, muss dieser als Bounding Box zu kachelnder Geometrien eingesetzt werden.
Hier zunächst die relevanten Parameter, exemplarisch für Zoomstufe 0:

In [51]:
custom_tms.cell_sizes[0]
custom_tms.tile_width
custom_tms.tile_height

4000.0

256

256

Die Tile limits lauten dann:

In [52]:
custom_tms.tile_limits(custom_extent, 0)

{'tileMinCol': 0,
 'tileMaxCol': 0,
 'tileMinRow': 0,
 'tileMaxRow': 0,
 'matrixWidth': 1,
 'matrixHeight': 1}

Und der envelope zur Kachel:

In [58]:
custom_envelope = custom_tms.tile_envelope(0,0,0)
custom_envelope

{'xmin': 2420000.0, 'ymin': 326000.0, 'xmax': 3444000.0, 'ymax': 1350000.0}

Wie erkennbar wird, entsteht hier eine quadratische Kachel, die deutlich über den Extent hinausragt:

In [60]:
envelope_width = custom_envelope["xmax"] - custom_envelope["xmin"]
envelope_height = custom_envelope["ymax"] - custom_envelope["ymin"]
envelope_width, envelope_height

(1024000.0, 1024000.0)

# TMS bei Vector Tiles

Alle bis hierhin geschilderten Zusammenhänge gelten zunächst für Rasterkacheln.
Wie sich eine TMS Definition auf die Visualisierung von Vector Tiles auswirkt, lässt die TMS Spec weitgehend unkommentiert.
Verwirrend ist in diesem Kontext vor allem die Definition von tileWidth und tileHeight als Pixelzahlen.
Denn anders als bei Rasterkacheln wird das komprimierter Binärformat (Protobuff), in dem Vector Tiles codiert sind, erst im Client gerendert! Nur so wird das nachträgliche Umstylen von Vector Tiles überhaupt möglich! Warum sollte man also im Vorfeld ein Pixelraster für jede einzelne Kachel festlegen? 

Ein aufschlussreicher Hinweis zum **extrapolated device grid** im Kontext von Vektordaten fällt in einer Randnotiz: *Some tiled vector data expressed in formats such as GeoJSON do not make use of an extrapolated device grid. Other tiled formats (e.g., MBTiles) define an internal coincident grid denser than the extrapolated device grid and express the position using indices in this denser grid instead of coordinates.*

Letzterer Satz bezieht sich zwar explizit auf MBTiles (abzugrenzen von Mapbox Vector Tiles). Allerdings trifft er auch auf die [Mapbox Vector Tiles Spezifikation](https://docs.mapbox.com/vector-tiles/specification/) zu. Auch hier werden geografische Koordinaten in ein "internal coincident grid" übertragen, was vor allem ihrer Komprimierung zu integer Werten dient. Im Grunde ist auch das eine Form von "Rasterisierung"! (*Bei genauerer Betrachtung sind in einer Datenbank gespeicherte Vektordaten letztlich auch Rasterdaten, da ein theoretisch kontinuierliches Koordinatensystem immer diskret wird, sobald eine Koordinate mit einer fixen Zahl an Nachkommastellen definiert wird*)

Dieses Grid ist in der Tat engmaschiger als das übliche Pixelraster, das bei den üblichen Parametern tileWidth = tileHeight = 256/512 Pixel entsteht. In der Spec fällt nämlich der Wert 4096. Die Rede ist von einem "extent", gemeint sind jedoch 4096 * 4096 Zellen. Also das "lokale" Koordinatensystem einer einzelnen Vektorkachel (zumindest nach meinem Verständnis). Da der Wert nur als Beispiel herangezogen wird, ist unklar ob er ein sinnigen Referenzwert darstellt bzw. wie man überhaupt zu einem "hinreichend engmaschigen" Grid gelangt. Die [PostGIS Implementierung der MVT Spezifikation](https://postgis.net/docs/ST_AsMVTGeom.html) hat diesen Wert als Default übernommen, leider ebenfalls ohne nähere Erläuterung. Womöglich ist dieser Wert bereits so engmaschig, dass man sich in der Praxis nicht mehr um die räumliche Auflösung sorgen muss.

Was jedoch klar wird: Die Zellengrößen, die ein TMS vorgibt, sind **UNABHÄNGIG** vom *internal coincident grid* einer einzelnen Vektorkachel.
Das TMS beeinflusst Vector Tiles nur insofern, als dass es die Größe der Kacheln (bzw. ihren Envelope) steuert.
* Je kleiner die im TMS definierte Zellengröße, desto mehr Kacheln fallen pro Zoomstufe an.
* Je mehr Kacheln anfallen, desto kürzer ihre Seitenlängen: die BBOX der Kachel bzw. der Tile Envelope schrumpft!
* Je kleiner die Kacheln, desto feiner die räumliche Auflösung. Nur, dass sich jede Kachel nicht mehr in tileWith * tileHeight unterteilt, sondern in das engmaschigere Gitter von 4096 * 4096 Zellen (sofern man den Default beibehält).

**Somit haben Vector Tiles stets eine größere räumliche Auflösung (bzw. kleinere Zellengrößen) als es das TMS suggeriert**

## Fazit

Auch wenn das TMS Konzept im Hinblick auf Rasterdaten entstandt und in Anwendung auf Vector Tiles Verwirrung stiften kann, bleibt es eine essenzielle Standardisierung zur Erzeugung und Darstellung gekachelter Geodaten. Nur bei bekanntem TMS kann der Client die richtigen Kacheln anfragen und an der richtigen Stelle rendern. Dazu muss er den hier illustrierten Pseudocode implementieren (tile limits und tile envelope). Dasselbe gilt für Software, die ausgehend Tile-Caches basierend auf einem TMS erzeugt.

Nicht alle Clients sind in dieser Hinsicht konsequent bzw. explizit. So setzt Mapbox ausschließlich auf WebMercatorQuad, setzt also einen impliziten Standard. Besondere Vorsicht ist auch bei QGis geboten: Die Berechnungen gehen hier pauschal vom TMS `GoogleCRS84Quad` aus.
In der allgemeinen Doku wird das leider nicht erwähnt, immerhin jedoch in der Dokumentation zur [QGIS API](https://qgis.org/api/3.20/classQgsVectorTileUtils.html). Lädt man dennoch einen Tile-Cache in WebMercatorQuad, werden die Kacheln trotzdem an richtiger Stelle positioniert.
Das funktioniert nur deshalb, weil beide TMS auf dasselbe "Bildpyramidenprinzip" zurückgreifen. Trotz unterschiedlicher Projektionen resultieren in diesem exaten Fall identische Maßstäbe und somit auch identische Kachedimensionen. Folglich zeigt eine Kachel mit den exemplarischen Indizes (x=0, y=1, z=1) sowohl in WebMercatorQuad als auch in GoogleCRS84Quad ein und denselben geografischen Ausschnitt!